In [209]:
import numpy as np
import pandas as pd
import requests
import urllib
import csv
from bs4 import BeautifulSoup
import re
import time
import datetime
import os
from functools import reduce
import matplotlib.pyplot as plt
try:
    from io import StringIO
except ImportError:
    from StringIO import StringIO

# Scrape historical stock data from Yahoo Finance

In [2]:
def get_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)

    return tickers

In [3]:
def crawl_data(ticker, start_time, end_time):
    env_url = 'https://finance.yahoo.com/quote/%s/history' % (ticker)
    r = requests.get(env_url)
    txt = r.content
    cookie = r.cookies['B']
    pattern = re.compile('.*"CrumbStore":\{"crumb":"(?P<crumb>[^"]+)"\}')

    for line in txt.splitlines():
        m = pattern.match(line.decode("utf-8"))
        if m is not None:
            crumb = m.groupdict()['crumb']

    start = datetime.datetime.strptime(start_time, '%Y/%m/%d').strftime("%s")
    end = datetime.datetime.strptime(end_time, '%Y/%m/%d').strftime("%s")

    download_url = "https://query1.finance.yahoo.com/v7/finance/download/%s?period1=%s&period2=%s&interval=1d&events=history&crumb=%s" \
                    % (ticker, start, end, crumb)
    data = requests.get(download_url, cookies={'B':cookie})

    # 200 and 201 for success
    if data.status_code != 200 and data.status_code != 201:
        return None
    print('{_ticker} successful!'.format(_ticker=ticker))
    table = pd.read_csv(StringIO(data.content.decode('utf8')))
    
    return table

In [4]:
def save_data(ticker_list, start_date, end_date):
    new_ticker_list = []
    for ticker in ticker_list:
        for i in range(3):
            table = crawl_data(ticker, start_date, end_date)
            if table is None:
                continue
            else:
                break
        else:
            continue
        table = pd.DataFrame(table[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']])
        
        # To ensure we have all 10 years' of data
        if pd.to_datetime(table['Date'][0]) > datetime.datetime(2008,1,2):
            continue
        
        table.to_csv('data/'+ticker+'.csv')
        if len(table) > 0:
            new_ticker_list.append(ticker)
    return new_ticker_list

In [5]:
ticker_list = get_tickers()#[0:10]
ticker_list = ticker_list + ["^GSPC"]

In [61]:
start_date = '2008/01/01'
end_date = '2017/12/31'
ticker_list = save_data(ticker_list, start_date, end_date)

MMM successful!
ABT successful!
ABBV successful!
ABMD successful!
ACN successful!
ATVI successful!
ADBE successful!
AMD successful!
AAP successful!
AES successful!
AET successful!
AMG successful!
AFL successful!
A successful!
APD successful!
AKAM successful!
ALK successful!
ALB successful!
ARE successful!
ALXN successful!
ALGN successful!
ALLE successful!
AGN successful!
ADS successful!
LNT successful!
ALL successful!
GOOGL successful!
GOOG successful!
MO successful!
AMZN successful!
AAL successful!
AEP successful!
AXP successful!
AIG successful!
AMT successful!
AWK successful!
ABC successful!
AME successful!
AMGN successful!
APH successful!
APC successful!
ADI successful!
ANDV successful!
ANSS successful!
ANTM successful!
AON successful!
AOS successful!
APA successful!
AIV successful!
AAPL successful!
AMAT successful!
APTV successful!
ADM successful!
ARNC successful!
AJG successful!
AIZ successful!
T successful!
ADSK successful!
ADP successful!
AZO successful!
AVB successful!
AVY succ

# Data Processing

## Stock data processing

Read data from local disk

In [429]:
ticker_list = []
for filename in os.listdir('/Users/luyang/Desktop/Invesco/data'):
    ticker = filename.split('.')[0]
    if len(ticker) > 0:
        ticker_list.append(ticker)
len(ticker_list)

453

Calculate returns and combine all the returns into a single data frame.

In [430]:
data_list = []
ticker_list_cur = ticker_list
for ticker in ticker_list_cur:
    data = pd.read_csv('data/' + ticker + '.csv', index_col=1)
    try:
        ret = pd.DataFrame(np.log(data['Adj Close'] / data['Adj Close'].shift()))
        ret.columns = [ticker]
        ret.index = data.index.values
        data_list.append(ret)
    except:
        print(ticker)
        ticker_list_cur.remove(ticker)
returns = reduce(lambda left, right: pd.merge(left, right, how = 'outer', left_index=True, right_index=True), data_list)

returns = returns.drop(returns.index[0])
returns = returns.dropna(1)
returns.index = pd.to_datetime(returns.index)
ticker_list_cur = returns.columns

BKNG
WELL
BHF


In [418]:
# from sklearn.preprocessing import StandardScaler
# volume_list = []
# ticker_list_cur = ticker_list
# for ticker in ticker_list_cur:
#     data = pd.read_csv('data/' + ticker + '.csv', index_col=1)
#     try:
#         vol = pd.DataFrame(data['Volume'])
#         vol.columns = [ticker]
#         vol.index = data.index.values
#         volume_list.append(vol)
#     except:
#         print(ticker)
#         ticker_list_cur.remove(ticker)
        
# volume = reduce(lambda left, right: 
#                 pd.merge(left, right, how = 'outer', left_index=True, right_index=True), volume_list)
# volume = volume.drop(volume.index[0])
# volume = volume.dropna(1)
# ticker_list_cur = returns.columns

# scaler = StandardScaler()
# scaler.fit(volume)
# volume = scaler.transform(volume)

## Factor data processing

Read the historical factor data saved in local drive.

Both versions of factors include Fama-Fench 5 factors (Market excess return, risk free return, SMB, HML, RMW, CMA), short-term reversal, long-term reversal, and momentum. 

Version 1 (factor_12) contains industry factors of 12 industries only, while version 2 (factor_49) contains 49 industry factors and public sentiment factor.

In [207]:
# version 1: 12 industry, no sentiment
factor_list = ['F-F_5_Factors', 'F-F_LT_Reversal_Factor_daily', 
               'F-F_Momentum_Factor_daily', 'F-F_ST_Reversal_Factor_daily',
               'Industry_12']

data_list = []
for factor in factor_list:
    data = pd.read_csv('factor/' + factor + '.csv', index_col=0)
    data.index = pd.to_datetime(data.index, format='%Y%m%d')
    data_list.append(data[(data.index>=datetime.datetime(2008,1,1)) & (data.index<=datetime.datetime(2017,12,31))])
    
factor_12 = reduce(lambda left,right: pd.merge(left,right,left_index=True,right_index=True), data_list)
factor_12 = factor_12.drop(factor_12.index[0])
factor_12.head()

,Mkt-RF,SMB,HML,RMW,CMA,RF,LT_Rev,Mom,ST_Rev,NoDur,...,Manuf,Enrgy,Chems,BusEq,Telcm,Utils,Shops,Hlth,Money,Other
2008-01-03,-0.03,-0.76,-0.40,-0.05,-0.44,0.01,-0.43,1.60,-0.98,0.60,...,0.41,0.34,0.27,-0.22,-0.09,0.20,-1.78,0.32,-0.45,-0.25
2008-01-04,-2.59,-0.43,0.28,-0.01,0.34,0.01,0.46,0.57,-0.42,-1.10,...,-2.97,-2.43,-1.52,-4.15,-1.96,-0.24,-2.52,-1.17,-2.70,-2.92
2008-01-07,0.02,-0.09,0.22,0.19,1.20,0.01,0.96,-0.52,0.05,1.98,...,-1.39,-1.04,0.12,-0.88,0.65,1.78,1.05,1.82,0.59,-0.47
2008-01-08,-1.69,-0.33,-0.96,0.20,-0.07,0.01,0.06,1.01,-1.43,-0.13,...,-2.43,-1.52,-1.04,-2.63,-2.65,-0.13,-1.94,0.93,-3.21,-1.98
2008-01-09,0.94,-0.73,-0.45,0.46,0.25,0.01,-0.08,0.25,-0.08,1.03,...,0.44,1.40,0.45,1.73,-0.15,1.29,0.78,1.61,1.32,0.68


In [431]:
# version 2: 49 industry, with sentiment
factor_list = ['F-F_5_Factors', 'F-F_LT_Reversal_Factor_daily', 
               'F-F_Momentum_Factor_daily', 'F-F_ST_Reversal_Factor_daily',
               'Industry_49']

data_list = []
for factor in factor_list:
    data = pd.read_csv('factor/' + factor + '.csv', index_col=0)
    data.index = pd.to_datetime(data.index, format='%Y%m%d')
    data_list.append(data[(data.index>=datetime.datetime(2008,1,1)) & (data.index<=datetime.datetime(2017,12,31))])
factor_49 = reduce(lambda left,right: pd.merge(left,right,left_index=True,right_index=True), data_list)

data = pd.read_csv('factor/AAII-AAII_SENTIMENT.csv', index_col=0)
data.index = pd.to_datetime(data.index, format='%m/%d/%y')
data = data[(data.index>=datetime.datetime(2008,1,1)) & (data.index<=datetime.datetime(2017,12,31))]
factor_49 = pd.merge(factor_49, data, how = 'outer',left_index=True,right_index=True)

factor_49[['Bullish','Neutral','Bearish']] = factor_49[['Bullish','Neutral','Bearish']].fillna(method = 'ffill')
factor_49 = factor_49.drop(factor_49.index[[0,1]])
factor_49.dropna()
returns = returns[returns.index.isin(factor_49.index)]
factor_49 = factor_49[factor_49.index.isin(returns.index)]

In [ ]:
# from sklearn.model_selection import TimeSeriesSplit

# tscv = TimeSeriesSplit(n_splits=2)
# for train_index, test_index in tscv.split(factor):
#     print("TRAIN:", train_index, "TEST:", test_index)

# Additional feature & Modeling implementation

### Feature

In [203]:
def add_lagged_returns(ticker, train_X, train_y, test_X, test_y):
    train_X_cur = train_X.copy()
    train_X_cur['lag'] = train_y[ticker].shift()
    test_X_cur = test_X.copy()
    test_X_cur['lag'] = test_y[ticker].shift()
    return train_X_cur.iloc[1:], train_y[ticker].iloc[1:], test_X_cur.iloc[1:], test_y[ticker].iloc[1:]

In [402]:
def add_volume(ticker, train_X, test_X):
    train_X_cur = train_X.copy()
    train_X_cur = pd.merge(train_X_cur, pd.DataFrame(vol), how = 'inner', 
                           left_index = True, right_index = True)
    test_X_cur = test_X.copy()
    test_X_cur = pd.merge(test_X_cur, pd.DataFrame(vol), how = 'inner', 
                          left_index = True, right_index = True)
    return train_X_cur, test_X_cur

### Model
Linear regression

In [201]:
from sklearn.linear_model import LinearRegression

def linear_regression(train_X, train_y, test_X, test_y):
    
    lin_reg = LinearRegression()
    lin_reg.fit(train_X, train_y)
    lin_reg.predict(test_X)
    
    return [lin_reg.score(train_X, train_y), lin_reg.score(test_X, test_y)]

Kernel regression

In [202]:
from sklearn.kernel_ridge import KernelRidge

def kernel_ridge(train_X, train_y, test_X, test_y, alpha, kernel):
    k_ridge = KernelRidge(alpha=alpha, kernel=kernel, degree=2, coef0=1)
    k_ridge.fit(train_X, train_y)
    k_ridge.predict(test_X)
    
    return [k_ridge.score(train_X, train_y), k_ridge.score(test_X, test_y)]

Random forest regression

Can provide us with information related to feature importance, so we can know which features can explain the returns the most.

In [474]:
from sklearn.ensemble import RandomForestRegressor

def rf_regressor(train_X, train_y, test_X, test_y, n):
    rfr = RandomForestRegressor(n_estimators = n, max_features = 'sqrt')
    rfr.fit(train_X, train_y)
    rfr.predict(test_X)
    
    return [[rfr.score(train_X, train_y), rfr.score(test_X, test_y)],
            rfr.feature_importances_]

# Model testing

In [ ]:
# train test split
def train_test_split(returns,factor,split_ratio):
    train_y = returns.iloc[0:int(len(returns) * split_ratio)]
    test_y = returns.iloc[int(len(returns) * split_ratio):]
    train_X = factor.iloc[0:int(len(returns) * split_ratio)]
    test_X = factor.iloc[int(len(returns) * split_ratio):]
    return train_X, train_y, test_X, test_y

### Factors only

Linear regression, 12 industry factor

In [381]:
train_X, train_y, test_X, test_y = train_test_split(returns,factor_12,0.8)
rsquared = {}
for ticker in ticker_list_cur:
    rsquared[ticker] = linear_regression(train_X, train_y[ticker], test_X, test_y[ticker])
pd.DataFrame(rsquared,index = ['train', 'test']).transpose().describe()

,train,test
count,446.000000,446.000000
mean,0.567950,0.330598
std,0.133713,0.244071
min,0.175267,-0.790004
25%,0.469069,0.206517
50%,0.576807,0.306683
75%,0.671102,0.472642
max,0.997742,0.995157


Kernel ridge regression, 12 industry factor

In [182]:
train_X, train_y, test_X, test_y = train_test_split(returns,factor_12,0.8)
kernel = 'poly'
for alpha in [1,5,10]:
    rsquared = {}
    print('alpha = ' + str(alpha))
    print(datetime.datetime.now())
    for ticker in ticker_list_cur:
        rsquared[ticker] = kernel_ridge(train_X, train_y[ticker], test_X, test_y[ticker], alpha, kernel)
    print(pd.DataFrame(rsquared,index = ['train', 'test']).transpose().describe())

alpha = 1
2018-06-23 11:00:53.696522
            train        test
count  446.000000  446.000000
mean     0.635753    0.293341
std      0.122884    0.259149
min      0.224539   -1.303237
25%      0.549525    0.169131
50%      0.647641    0.282797
75%      0.730984    0.448000
max      0.998796    0.994280
alpha = 5
2018-06-23 11:03:09.592714
            train        test
count  446.000000  446.000000
mean     0.613500    0.329285
std      0.126575    0.238689
min      0.206318   -0.678773
25%      0.519543    0.210302
50%      0.625793    0.304941
75%      0.710379    0.474461
max      0.998631    0.994021
alpha = 10
2018-06-23 11:05:21.123292
            train        test
count  446.000000  446.000000
mean     0.603335    0.336836
std      0.127869    0.232724
min      0.199204   -0.613916
25%      0.510031    0.216668
50%      0.616180    0.312081
75%      0.701874    0.476863
max      0.998460    0.993367


Kernel ridge regression, 49 industry factor

In [442]:
train_X, train_y, test_X, test_y = train_test_split(returns,factor_49,0.8)
kernel = 'poly'
for alpha in [1,5,10,20]:
    rsquared = {}
    print('alpha = ' + str(alpha))
    print(datetime.datetime.now())
    for ticker in ticker_list_cur:
        rsquared[ticker] = kernel_ridge(train_X, train_y[ticker], test_X, test_y[ticker], alpha, kernel)
    print(pd.DataFrame(rsquared,index = ['train', 'test']).transpose().describe())

alpha = 1
2018-06-23 15:35:17.047578
            train        test
count  446.000000  446.000000
mean     0.802071    0.317135
std      0.089073    0.263115
min      0.500453   -1.322124
25%      0.746287    0.164006
50%      0.816031    0.310747
75%      0.871594    0.487351
max      0.999372    0.991326
alpha = 5
2018-06-23 15:37:34.314037
            train        test
count  446.000000  446.000000
mean     0.741909    0.375095
std      0.107389    0.229225
min      0.400441   -0.594557
25%      0.671355    0.230463
50%      0.760016    0.365790
75%      0.824539    0.537164
max      0.998934    0.989905
alpha = 10
2018-06-23 15:39:52.066300
            train        test
count  446.000000  446.000000
mean     0.714899    0.382100
std      0.113039    0.221576
min      0.356668   -0.461789
25%      0.642475    0.239722
50%      0.731962    0.377556
75%      0.801016    0.541088
max      0.998502    0.988072
alpha = 20
2018-06-23 15:42:14.948573
            train        test
count  446

### With lagged return

Linear regression, 12 industry factor

In [294]:
train_X, train_y, test_X, test_y = train_test_split(returns,factor_12,0.8)
rsquared = {}
for ticker in ticker_list_cur:
    train_X_cur, train_y_cur, test_X_cur, test_y_cur = add_lagged_returns(ticker, train_X, train_y, test_X, test_y)
    rsquared[ticker] = linear_regression(train_X_cur, train_y_cur, test_X_cur, test_y_cur)
    
pd.DataFrame(rsquared,index = ['train', 'test']).transpose().describe()

,train,test
count,446.000000,446.000000
mean,0.569963,0.330161
std,0.133940,0.241448
min,0.176550,-0.788999
25%,0.472049,0.205782
50%,0.579847,0.306414
75%,0.672043,0.474027
max,0.997766,0.995217


In [295]:
# train_X, train_y, test_X, test_y = train_test_split(returns,factor_12,0.8)
# rsquared = {}
# for ticker in ticker_list_cur:
#     train_X_cur, train_y_cur, test_X_cur, test_y_cur = add_lagged_returns(ticker, train_X, train_y, test_X, test_y)
#     train_X_cur, test_X_cur = add_volume(ticker, train_X_cur, test_X_cur)
#     rsquared[ticker] = linear_regression(train_X_cur, train_y_cur, test_X_cur, test_y_cur)

# pd.DataFrame(rsquared,index = ['train', 'test']).transpose().describe()

,train,test
count,446.000000,446.000000
mean,0.573472,0.326275
std,0.131524,0.244194
min,0.177544,-0.789711
25%,0.475904,0.196766
50%,0.582108,0.301577
75%,0.672667,0.481310
max,0.997843,0.995187


Kernel ridge regression, 12 industry factor

In [405]:
from sklearn.preprocessing import normalize
train_X, train_y, test_X, test_y = train_test_split(returns,factor_12,0.8)

kernel = 'poly'
for alpha in [1,5,10]:
    rsquared = {}
    print('alpha = ' + str(alpha))
    print(datetime.datetime.now())
    for ticker in ticker_list_cur:
        train_X_cur, train_y_cur, test_X_cur, test_y_cur = add_lagged_returns(ticker, train_X, train_y, test_X, test_y)
#         train_X_cur, test_X_cur = add_volume(ticker, train_X_cur, test_X_cur)
#         train_X_cur = normalize(train_X_cur)
#         test_X_cur = normalize(test_X_cur)
        rsquared[ticker] = kernel_ridge(train_X_cur, train_y_cur, test_X_cur, test_y_cur, alpha, kernel)
    print(pd.DataFrame(rsquared,index = ['train', 'test']).transpose().describe())
    
kernel = 'rbf'
rsquared = {}
print('kernel = ' + str(kernel))
print(datetime.datetime.now())
for ticker in ticker_list_cur:
    train_X_cur, train_y_cur, test_X_cur, test_y_cur = add_lagged_returns(ticker, train_X, train_y, test_X, test_y)
    train_X_cur, test_X_cur = add_volume(ticker, train_X_cur, test_X_cur)
    rsquared[ticker] = kernel_ridge(train_X, train_y[ticker], test_X, test_y[ticker], alpha, kernel)
print(pd.DataFrame(rsquared,index = ['train', 'test']).transpose().describe())

alpha = 1
2018-06-23 14:04:34.264018
            train        test
count  446.000000  446.000000
mean     0.635269    0.296292
std      0.123051    0.256271
min      0.223977   -1.221721
25%      0.549360    0.174283
50%      0.646864    0.282477
75%      0.731524    0.448752
max      0.998791    0.994276
alpha = 5
2018-06-23 14:06:40.674458
            train        test
count  446.000000  446.000000
mean     0.612570    0.330028
std      0.126745    0.237885
min      0.205570   -0.646931
25%      0.518474    0.210527
50%      0.624626    0.306389
75%      0.709937    0.474824
max      0.998621    0.993986
alpha = 10
2018-06-23 14:08:46.099290
            train        test
count  446.000000  446.000000
mean     0.602336    0.337014
std      0.128018    0.232181
min      0.198451   -0.611187
25%      0.508844    0.216103
50%      0.615084    0.311053
75%      0.701495    0.477108
max      0.998440    0.993289
kernel = rbf
2018-06-23 14:10:50.923900
            train        test
count  4

Kernel ridge regression, 49 industry factor

In [413]:
train_X, train_y, test_X, test_y = train_test_split(returns,factor_49,0.8)
kernel = 'poly'
for alpha in [1,5,10]:
    rsquared = {}
    print('alpha = ' + str(alpha))
    print(datetime.datetime.now())
    for ticker in ticker_list_cur:
        train_X_cur, train_y_cur, test_X_cur, test_y_cur = add_lagged_returns(ticker, train_X, train_y, test_X, test_y)

        train_y_cur = train_y_cur[train_y_cur.index.isin(train_X_cur.index)]
        test_y_cur = test_y_cur[test_y_cur.index.isin(test_X_cur.index)]
        rsquared[ticker] = kernel_ridge(train_X_cur, train_y_cur, 
                                        test_X_cur, test_y_cur, alpha, kernel)
    print(pd.DataFrame(rsquared,index = ['train', 'test']).transpose().describe())
    


alpha = 1
2018-06-23 14:19:37.019940
            train        test
count  446.000000  446.000000
mean     0.800922    0.318512
std      0.089502    0.261748
min      0.498691   -1.303086
25%      0.744980    0.168865
50%      0.814976    0.311134
75%      0.870991    0.490568
max      0.999366    0.991298
alpha = 5
2018-06-23 14:21:59.489338
            train        test
count  446.000000  446.000000
mean     0.740712    0.375264
std      0.107724    0.228880
min      0.398198   -0.585845
25%      0.669605    0.230249
50%      0.758692    0.364022
75%      0.823678    0.536517
max      0.998922    0.989851
alpha = 10
2018-06-23 14:24:27.785519
            train        test
count  446.000000  446.000000
mean     0.713686    0.381885
std      0.113318    0.221415
min      0.354621   -0.462517
25%      0.640755    0.239714
50%      0.730641    0.378239
75%      0.800101    0.541125
max      0.998484    0.987988


Kernel ridge regression with rbf kernel, 49 industry factor

In [415]:
train_X, train_y, test_X, test_y = train_test_split(returns,factor_49,0.8)
kernel = 'rbf'

rsquared = {}
print('alpha = ' + str(alpha))
print(datetime.datetime.now())
for ticker in ticker_list_cur:
    train_X_cur, train_y_cur, test_X_cur, test_y_cur = add_lagged_returns(ticker, train_X, train_y, test_X, test_y)
#       train_X_cur, test_X_cur = add_volume(ticker, train_X_cur, test_X_cur)
    train_y_cur = train_y_cur[train_y_cur.index.isin(train_X_cur.index)]
    test_y_cur = test_y_cur[test_y_cur.index.isin(test_X_cur.index)]
    rsquared[ticker] = kernel_ridge(train_X_cur, train_y_cur, 
                                    test_X_cur, test_y_cur, alpha, kernel)
print(pd.DataFrame(rsquared,index = ['train', 'test']).transpose().describe())
    

alpha = 10
2018-06-23 14:27:53.020860
            train        test
count  446.000000  446.000000
mean     0.342091    0.283530
std      0.054724    0.125364
min      0.194416   -0.018701
25%      0.303372    0.187738
50%      0.340758    0.268493
75%      0.381295    0.368364
max      0.569612    0.768757


Random forest with 49 industry factor, control max_features for the purpose of regularization.

In [441]:
train_X, train_y, test_X, test_y = train_test_split(returns,factor_49,0.8)

n_estimators = [100]
for n in n_estimators:
    print('n_estimators = ' + str(n))
    print(datetime.datetime.now())
    rsquared = {}
    feature_importance = {}
    for ticker in ticker_list_cur:
        train_X_cur, train_y_cur, test_X_cur, test_y_cur = add_lagged_returns(ticker, train_X, train_y, test_X, test_y)
        train_y_cur = train_y_cur[train_y_cur.index.isin(train_X_cur.index)]
        test_y_cur = test_y_cur[test_y_cur.index.isin(test_X_cur.index)]
        result = rf_regressor(train_X_cur, train_y_cur, test_X_cur, test_y_cur, n)
        rsquared[ticker] = result[0]
        feature_importance[ticker] = result[1]
    print(pd.DataFrame(rsquared,index = ['train', 'test']).transpose().describe())
    
print(datetime.datetime.now())

n_estimators = 100
2018-06-23 15:22:32.384853
            train        test
count  446.000000  446.000000
mean     0.935966    0.373600
std      0.018227    0.186442
min      0.883760   -0.148818
25%      0.923227    0.246016
50%      0.938831    0.366209
75%      0.949617    0.503974
max      0.996545    0.974885
2018-06-23 15:31:32.932491


Random forest parameter tuning (n_estimators)

In [475]:
train_X, train_y, test_X, test_y = train_test_split(returns,factor_49,0.8)

n_estimators = [50,100,200]
for n in n_estimators:
    print('n_estimators = ' + str(n))
    print(datetime.datetime.now())
    rsquared = {}
    feature_importance = {}
    for ticker in ticker_list_cur:
        train_X_cur, train_y_cur, test_X_cur, test_y_cur = add_lagged_returns(ticker, train_X, train_y, test_X, test_y)
        train_y_cur = train_y_cur[train_y_cur.index.isin(train_X_cur.index)]
        test_y_cur = test_y_cur[test_y_cur.index.isin(test_X_cur.index)]
        result = rf_regressor(train_X_cur, train_y_cur, test_X_cur, test_y_cur, n)
        rsquared[ticker] = result[0]
#         feature_importance[ticker] = result[1]
    print(pd.DataFrame(rsquared,index = ['train', 'test']).transpose().describe())
    
print(datetime.datetime.now())

n_estimators = 50
2018-06-23 16:09:41.440913
            train        test
count  446.000000  446.000000
mean     0.932784    0.364296
std      0.019304    0.189125
min      0.862542   -0.387404
25%      0.918915    0.227681
50%      0.935474    0.355004
75%      0.947418    0.493885
max      0.996127    0.972774
n_estimators = 100
2018-06-23 16:14:12.565493
            train        test
count  446.000000  446.000000
mean     0.935855    0.373508
std      0.018367    0.187219
min      0.882531   -0.148344
25%      0.922359    0.240936
50%      0.938809    0.362633
75%      0.949919    0.503843
max      0.996677    0.976096
n_estimators = 200
2018-06-23 16:23:14.778219
            train        test
count  446.000000  446.000000
mean     0.937355    0.378467
std      0.017763    0.186842
min      0.883523   -0.139370
25%      0.924767    0.247781
50%      0.940186    0.369259
75%      0.950550    0.509790
max      0.996817    0.976705
2018-06-23 16:41:36.636967


### Feature Importance

In [471]:
importance = pd.DataFrame(feature_importance).transpose()
importance.columns = list(factor_49.columns.values)+['lag']
importance.describe().transpose().sort_values('mean')

,count,mean,std,min,25%,50%,75%,max
RF,446.0,0.000914,0.000688,0.000083,0.000513,0.000711,0.001065,0.006074
Bearish,446.0,0.005720,0.002076,0.000343,0.004116,0.005537,0.006795,0.013686
Bullish,446.0,0.006319,0.002484,0.000429,0.004742,0.005898,0.007508,0.028427
Neutral,446.0,0.006679,0.002258,0.000359,0.005127,0.006245,0.007732,0.015712
CMA,446.0,0.008035,0.003518,0.000616,0.005661,0.007444,0.009300,0.040222
SMB,446.0,0.008155,0.002947,0.001056,0.006010,0.007652,0.009736,0.022360
LT_Rev,446.0,0.008256,0.002899,0.000583,0.006430,0.007752,0.009606,0.021780
Agric,446.0,0.008260,0.005639,0.000729,0.005921,0.007568,0.009629,0.086641
Gold,446.0,0.008454,0.018723,0.000649,0.005738,0.007163,0.008989,0.398195
ST_Rev,446.0,0.008786,0.003026,0.000830,0.006832,0.008383,0.010219,0.027730


# Project summary & Next steps

This project is a general research on the factors explaining stock return. I have included all the S&P500 stocks here except the ones without complete data in the period 2008/1/1 to 2017/12/31.

I started from linear regression and using only the factors as explaining variable. The performance was not very good with high level of overfitting. Therefore, I tried kernel ridge regression with more features (more industry categories and sentiment factor), as ridge regression can relieve the problem of overfitting while the kernel can capture non-linear relationship between the factors and the stock returns. 

However, these two models cannot give us any information on feature importance, which we would like to have in order to understand how the features can explain the returns. Therefore, I further tested random forest with different number of estimators. We can see that although still suffering from overfitting, random forest regression can give us the highest training and testing R-squared among all the models. The average R-squared in training set is as high as 93.5%, that means the factors we have found can indeed explain the returns to a very large extent, however, the parameters are not persistent, which result in the poor performance in the testing set. 

According to the feature importance, the most important feature is market excess return, while others including individual industry performance. The traditionally recognized factors like SMB, HML and MOM are not very important here. This may due to the fact that we are trying to use general market factor to explain individual stock performance. Therefore, if we can calculate stock-specific factors, they may have more explaining power. 

Next steps:
1. Include stock return moving average instead of lagged return as a factor.
2. Generate more stock-specific factors.
3. Try other models and regularization methods.